In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D   # noqa: F401 (für 3‑D‑Plots)

def triple_scan(psi_range: tuple[float, float],
                N: int,
                k_range: tuple[float, float],
                theta_range: tuple[float, float],
                psi_steps: int = 30,
                k_steps: int = 80,
                theta_steps: int = 70,
                A: float = 1.0,
                eps1: float = 1e-6,
                eps2: float = 1e-6):
    """
    Scan über ψ, k und θ.
    Rückgabe: 3‑D‑Arrays für E1, E2 und eine boolesche Maske, die angibt,
    wo beide Fehlermesswerte unter den Toleranzen liegen.
    """
    # 1️⃣ Gitter erzeugen
    psis   = np.linspace(*psi_range,   psi_steps)
    ks     = np.linspace(*k_range,     k_steps)
    thetas = np.linspace(*theta_range, theta_steps)

    # 2️⃣ Ergebnis‑Arrays (psi × theta × k)
    E1_grid = np.empty((psi_steps, theta_steps, k_steps))
    E2_grid = np.empty((psi_steps, theta_steps, k_steps))
    valid   = np.zeros_like(E1_grid, dtype=bool)

    # 3️⃣ Hauptschleifen
    for ip, psi in enumerate(psis):
        # Schrittweite und Periode für diesen psi
        dx   = (2 * np.pi / psi) / N
        L_xi = 2 * np.pi / psi
        xi   = np.linspace(0, L_xi, N + 1)   # identisch für alle k,θ

        for it, th in enumerate(thetas):
            # Sinus‑Term (gleiche für alle k bei festem θ)
            sin_term = A * np.sin(th * xi)    # shape (N+1,)

            for ik, k in enumerate(ks):
                chi = k * xi + sin_term        # χ_n mit Phasen‑Modulation

                # Fehlermesswerte
                E1 = abs(xi[-1] - L_xi)        # nur von ψ abhängig
                E2 = abs((chi[-1] - chi[0]) / (xi[-1] - xi[0]) - k)

                E1_grid[ip, it, ik] = E1
                E2_grid[ip, it, ik] = E2
                valid[ip, it, ik]   = (E1 < eps1) and (E2 < eps2)

    return psis, ks, thetas, E1_grid, E2_grid, valid


# -------------------- Ausführung --------------------
psi_min, psi_max = 1.0, 10.0
k_min,   k_max   = 0.0,   1.0
theta_min, theta_max = 0.0, 20.0
N_val = 4000

psis, ks, thetas, E1, E2, ok = triple_scan(
    psi_range=(psi_min, psi_max),
    N=N_val,
    k_range=(k_min, k_max),
    theta_range=(theta_min, theta_max),
    psi_steps=40,
    k_steps=120,
    theta_steps=100,
    A=1.0,
    eps1=1e-6,
    eps2=1e-6,
)

# -------------------- 1️⃣ JSON‑Export --------------------
output_json = {
    "metadata": {
        "description": "Triple scan over psi, k, theta (v5 core)",
        "psi_range": [psi_min, psi_max],
        "k_range":   [k_min,   k_max],
        "theta_range": [theta_min, theta_max],
        "N_steps_per_period": N_val,
        "amplitude_A": 1.0,
        "tolerance_E1": 1e-6,
        "tolerance_E2": 1e-6
    },
    # Wir speichern die Gitterpunkte als Listen (JSON‑kompatibel)
    "psi_values":  psis.tolist(),
    "k_values":    ks.tolist(),
    "theta_values": thetas.tolist(),

    # Fehlermesswerte – 3‑D‑Arrays flachlegen, um JSON‑Kompatibilität zu wahren
    "E1": E1.ravel().tolist(),
    "E2": E2.ravel().tolist(),
    "valid": ok.ravel().tolist(),

    # Zusätzlich die Shape‑Information, damit wir beim Laden das Grid wieder rekonstruieren können
    "grid_shape": E1.shape   # (len(psi), len(theta), len(k))
}

# Schreibe die Datei
json_path = "triple_scan_results.json"
with open(json_path, "w") as fp:
    json.dump(output_json, fp, indent=2)

print(f"✅ Ergebnisse wurden nach '{json_path}' geschrieben.")


# -------------------- 2️⃣ Alternativ: CSV‑/TXT‑Export --------------------
# Manchmal ist ein flaches Tabellenformat praktischer, besonders wenn du
# die Daten in Excel, pandas oder ähnliches laden willst.
csv_path = "triple_scan_results.txt"
with open(csv_path, "w") as fp:
    # Header‑Zeile
    fp.write("psi,k,theta,E1,E2,valid\n")
    # Alle Kombinationen iterieren
    for i_psi, psi in enumerate(psis):
        for i_th, th in enumerate(thetas):
            for i_k, k in enumerate(ks):
                idx = (i_psi, i_th, i_k)
                line = f"{psi},{k},{th},{E1[idx]:.6e},{E2[idx]:.6e},{int(ok[idx])}\n"
                fp.write(line)

print(f"✅ Tabellarische Ausgabe wurde nach '{csv_path}' geschrieben.")


✅ Ergebnisse wurden nach 'triple_scan_results.json' geschrieben.
✅ Tabellarische Ausgabe wurde nach 'triple_scan_results.txt' geschrieben.


In [2]:
"""
psi_spectrum.py

Lädt das Ergebnis eines ψ‑θ‑k‑Scans (triple_scan_results.json) und
wertet **alle ψ‑Werte** aus:

* für jeden ψ‑Slice (fixierter θ) wird eine lineare Regression
  k ↔ Fehlermesswert (E2 oder E1) durchgeführt,
* die Fit‑Parameter, R² und die Anzahl gültiger Punkte werden
  in einer CSV‑ und einer aggregierten JSON‑Datei gespeichert,
* optional wird für jeden ψ‑Wert ein Plot (Scatter + Fit) erzeugt.
"""

import json
import csv
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt


# ----------------------------------------------------------------------
# 1️⃣  Hilfsfunktionen
# ----------------------------------------------------------------------
def load_triple_json(path: Path):
    """Lädt das JSON‑Ergebnis eines ψ‑θ‑k‑Scans und gibt 3‑D‑Arrays zurück."""
    with path.open("r", encoding="utf-8") as fp:
        data = json.load(fp)

    shape = tuple(data["grid_shape"])          # (len(psi), len(theta), len(k))
    psi_vals   = np.array(data["psi_values"])
    theta_vals = np.array(data["theta_values"])
    k_vals     = np.array(data["k_values"])

    E1 = np.array(data["E1"]).reshape(shape)
    E2 = np.array(data["E2"]).reshape(shape)
    valid = np.array(data["valid"]).reshape(shape)

    return {
        "psi":   psi_vals,
        "theta": theta_vals,
        "k":     k_vals,
        "E1":    E1,
        "E2":    E2,
        "valid": valid,
        "metadata": data.get("metadata", {})
    }


def linear_fit(x, y, degree: int = 1):
    """
    Fit eines Polynoms vom Grad `degree`.
    Rückgabe: (Koeffizienten, Vorhersage‑Array, R²)
    """
    coeff = np.polyfit(x, y, deg=degree)
    pred  = np.polyval(coeff, x)

    # Bestimmtheitsmaß R²
    ss_res = np.sum((y - pred) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else float("nan")
    return coeff, pred, r2


def plot_psi_slice(k_vals, err_vals, fit_vals, coeff, r2,
                   psi_val, theta_val, target_error,
                   out_dir: Path, suffix: str = "") -> Path:
    """Erzeugt einen Scatter‑Plot + Fit‑Linie für einen einzelnen ψ‑Slice."""
    plt.figure(figsize=(8, 4.5))
    plt.scatter(k_vals, err_vals, s=12, color="steelblue",
                label="gültige Messwerte")
    plt.plot(k_vals, fit_vals, color="crimson", linewidth=2,
             label=f"Fit (R²={r2:.3f})")
    plt.title(f"{target_error} vs. k  (ψ={psi_val:.2f}, θ={theta_val:.2f})")
    plt.xlabel("Steigung k")
    plt.ylabel(target_error)
    plt.legend()
    plt.grid(True, ls=":", alpha=0.6)
    plt.tight_layout()

    filename = out_dir / f"psi_{psi_val:.2f}{suffix}.png"
    plt.savefig(filename, dpi=300)
    plt.close()
    return filename


# ----------------------------------------------------------------------
# 2️⃣  Hauptfunktion – Analyse über das gesamte ψ‑Spektrum
# ----------------------------------------------------------------------
def analyse_psi_spectrum(
    json_input_path: Path,
    output_dir: Path,
    target_error: str = "E2",          # "E1" oder "E2"
    theta_index: int = 0,              # welcher θ‑Slice (0 = erster)
    poly_degree: int = 1,              # 1 = linear, 2 = quadratisch, …
    create_plots: bool = True
):
    """
    - Lädt das Triple‑Scan‑JSON.
    - Iteriert über **alle ψ‑Werte**.
    - Für jeden ψ‑Slice (fixierter θ) wird eine Regression k ↔ error durchgeführt.
    - Ergebnisse werden in einer aggregierten JSON‑Datei, einer CSV‑Tabelle
      und optional als einzelne PNG‑Plots gespeichert.
    """
    output_dir.mkdir(parents=True, exist_ok=True)

    # ----- Daten laden -----
    data = load_triple_json(json_input_path)

    psi_vals   = data["psi"]
    theta_vals = data["theta"]
    k_vals     = data["k"]
    E1         = data["E1"]
    E2         = data["E2"]
    valid      = data["valid"]

    theta_val = theta_vals[theta_index]

    # ----- Ergebnis‑Container -----
    aggregated = {
        "metadata": data["metadata"],
        "target_error": target_error,
        "theta_fixed": float(theta_val),
        "theta_index": int(theta_index),
        "poly_degree": int(poly_degree),
        "psi_results": []          # Liste von Dicts, je ψ
    }

    # ----- CSV‑Header -----
    csv_path = output_dir / "psi_spectrum_summary.csv"
    csv_file = csv_path.open("w", newline="", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow([
        "psi", "theta", "poly_degree",
        "slope", "intercept", "r_squared",
        "num_valid_points"
    ])

    # ----- Schleife über alle ψ -----
    for i_psi, psi_val in enumerate(psi_vals):
        # Fehlermesswert‑Slice für diesen ψ und den gewählten θ
        err_slice = E1[i_psi, theta_index, :] if target_error.upper() == "E1" else E2[i_psi, theta_index, :]
        valid_slice = valid[i_psi, theta_index, :]

        # Nur gültige Punkte für den Fit verwenden
        k_valid   = k_vals[valid_slice]
        err_valid = err_slice[valid_slice]

        if len(k_valid) < 2:
            # Nicht genug Punkte – überspringen
            print(f"⚠️ ψ={psi_val:.2f}: nur {len(k_valid)} gültige Punkte → übersprungen.")
            continue

        # ----- Regression -----
        coeff, fit_vals, r2 = linear_fit(k_valid, err_valid, degree=poly_degree)

        # ----- Ergebnis‑Dict für diesen ψ -----
        result = {
            "psi": float(psi_val),
            "theta": float(theta_val),
            "poly_degree": int(poly_degree),
            "coefficients": coeff.tolist(),   # [a_n, ..., a_0]
            "r_squared": float(r2),
            "num_valid_points": int(len(k_valid))
        }
        aggregated["psi_results"].append(result)

        # ----- CSV‑Zeile -----
        if poly_degree == 1:
            slope, intercept = coeff
            csv_writer.writerow([psi_val, theta_val, poly_degree,
                                 slope, intercept, r2, len(k_valid)])
        else:
            # Für höhere Grade schreiben wir die Koeffizienten als String
            csv_writer.writerow([psi_val, theta_val, poly_degree,
                                 ";".join(map(str, coeff)), "", r2, len(k_valid)])

        # ----- Plot (optional) -----
        if create_plots:
            plot_path = plot_psi_slice(
                k_vals=k_valid,
                err_vals=err_valid,
                fit_vals=fit_vals,
                coeff=coeff,
                r2=r2,
                psi_val=psi_val,
                theta_val=theta_val,
                target_error=target_error,
                out_dir=output_dir,
                suffix="_fit"
            )
            print(f"✅ Plot für ψ={psi_val:.2f} → {plot_path}")

    # ----- Abschluss -----
    csv_file.close()
    print(f"\n📊 CSV‑Zusammenfassung geschrieben nach {csv_path}")

    json_out = output_dir / "psi_spectrum_aggregated.json"
    with json_out.open("w", encoding="utf-8") as fp:
        json.dump(aggregated, fp, indent=2, ensure_ascii=False)
    print(f"📁 Aggregierte Ergebnisse JSON geschrieben nach {json_out}")

    return aggregated


# ----------------------------------------------------------------------
# 3️⃣  Skript‑Entry‑Point
# ----------------------------------------------------------------------
if __name__ == "__main__":
    # Pfade – bei Bedarf anpassen
    INPUT_JSON   = Path("triple_scan_results.json")   # <‑‑ hier liegt deine Scan‑Datei
    OUTPUT_DIR   = Path("psi_spectrum_output")        # Ergebnis‑Ordner

    # Parameter – du kannst sie nach Belieben ändern
    TARGET_ERROR = "E2"      # "E1" oder "E2"
    THETA_IDX    = 0         # welcher θ‑Slice (0 = erstes Element)
    POLY_DEGREE = 1         # 1 = linearer Fit
    CREATE_PLOTS = True     # False, wenn du nur CSV/JSON willst

    analyse_psi_spectrum(
        json_input_path=INPUT_JSON,
        output_dir=OUTPUT_DIR,
        target_error=TARGET_ERROR,
        theta_index=THETA_IDX,
        poly_degree=POLY_DEGREE,
        create_plots=CREATE_PLOTS
    )




✅ Plot für ψ=1.00 → psi_spectrum_output\psi_1.00_fit.png
✅ Plot für ψ=1.23 → psi_spectrum_output\psi_1.23_fit.png
✅ Plot für ψ=1.46 → psi_spectrum_output\psi_1.46_fit.png
✅ Plot für ψ=1.69 → psi_spectrum_output\psi_1.69_fit.png
✅ Plot für ψ=1.92 → psi_spectrum_output\psi_1.92_fit.png
✅ Plot für ψ=2.15 → psi_spectrum_output\psi_2.15_fit.png
✅ Plot für ψ=2.38 → psi_spectrum_output\psi_2.38_fit.png
✅ Plot für ψ=2.62 → psi_spectrum_output\psi_2.62_fit.png
✅ Plot für ψ=2.85 → psi_spectrum_output\psi_2.85_fit.png
✅ Plot für ψ=3.08 → psi_spectrum_output\psi_3.08_fit.png
✅ Plot für ψ=3.31 → psi_spectrum_output\psi_3.31_fit.png
✅ Plot für ψ=3.54 → psi_spectrum_output\psi_3.54_fit.png
✅ Plot für ψ=3.77 → psi_spectrum_output\psi_3.77_fit.png
✅ Plot für ψ=4.00 → psi_spectrum_output\psi_4.00_fit.png
✅ Plot für ψ=4.23 → psi_spectrum_output\psi_4.23_fit.png
✅ Plot für ψ=4.46 → psi_spectrum_output\psi_4.46_fit.png
✅ Plot für ψ=4.69 → psi_spectrum_output\psi_4.69_fit.png
✅ Plot für ψ=4.92 → psi_spectru

In [3]:
"""
psi_analysis_1.py

- Lädt das aggregierte Ergebnis eines ψ‑θ‑k‑Scans (psi_spectrum_aggregated.json).
- Entfernt NaN‑Einträge.
- Plottet:
    * R²‑Scatter über das gesamte ψ‑Spektrum
    * K‑Means‑Clustering (2 Cluster: "hoch‑qualitativ" vs. "weniger‑qualitativ")
- Speichert sämtliche Plots automatisch in ./analysis_output/plots/
- Exportiert die numerischen Resultate in:
    * CSV (leicht zu öffnen in Excel/Sheets)
    * JSON (strukturierte Daten für weitere Verarbeitung)
"""

import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
import csv

# -------------------------------------------------
# 1️⃣  Pfade & Einstellungen
# -------------------------------------------------
INPUT_JSON   = Path("psi_spectrum_output/psi_spectrum_aggregated.json")   # <- dein Scan‑Ergebnis
OUT_ROOT     = Path("analysis_output")                # Haupt‑Ausgabeverzeichnis
PLOTS_DIR    = OUT_ROOT / "plots"
CSV_OUT      = OUT_ROOT / "psi_results.csv"
JSON_OUT     = OUT_ROOT / "psi_results.json"

# Erstelle die Ordner, falls sie noch nicht existieren
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------------------------------
# 2️⃣  Daten laden
# -------------------------------------------------
with INPUT_JSON.open("r", encoding="utf-8") as fp:
    agg = json.load(fp)

psi_vals   = np.array([entry["psi"] for entry in agg["psi_results"]])
r2_vals    = np.array([entry["r_squared"] for entry in agg["psi_results"]])
slope_vals = np.array([entry["coefficients"][0] for entry in agg["psi_results"]])
inter_vals = np.array([entry["coefficients"][1] for entry in agg["psi_results"]])

# -------------------------------------------------
# 3️⃣  NaNs entfernen (kritisch für K‑Means)
# -------------------------------------------------
valid_mask = np.isfinite(psi_vals) & np.isfinite(r2_vals)
if not np.any(valid_mask):
    raise RuntimeError("Alle Daten enthalten NaN – nichts zu verarbeiten.")

psi_clean   = psi_vals[valid_mask]
r2_clean    = r2_vals[valid_mask]
slope_clean = slope_vals[valid_mask]
inter_clean = inter_vals[valid_mask]

print(f"Ursprünglich {len(psi_vals)} Einträge → {len(psi_clean)} gültige (ohne NaN).")

# -------------------------------------------------
# 4️⃣  Plot 1: R²‑Scatter über ψ
# -------------------------------------------------
plt.figure(figsize=(8, 4.5))
plt.scatter(psi_vals, r2_vals, s=12, c="steelblue", label="Alle (inkl. NaN‑Markierungen)")
plt.scatter(psi_clean, r2_clean, s=30, c="orange", label="Nur gültige")
threshold = 0.999999
plt.axhline(threshold, color="red", ls="--", lw=1, label=f"R²‑Schwelle {threshold}")
plt.xlabel(r"$\psi$")
plt.ylabel(r"$R^2$")
plt.title("Fit‑Qualität (R²) über das ψ‑Spektrum")
plt.legend()
plt.grid(True, ls=":", alpha=0.5)
plt.tight_layout()
r2_plot_path = PLOTS_DIR / "r2_scatter.png"
plt.savefig(r2_plot_path, dpi=300)
plt.close()
print(f"✅ Plot R²‑Scatter gespeichert → {r2_plot_path}")

# -------------------------------------------------
# 5️⃣  K‑Means‑Clustering (2 Cluster)
# -------------------------------------------------
if len(psi_clean) > 2:                     # mindestens 2 Punkte nötig
    X = np.column_stack((psi_clean, r2_clean))
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
    labels = kmeans.labels_

    # Plot des Clusters
    plt.figure(figsize=(8, 4.5))
    for lab, col in zip([0, 1], ["steelblue", "orange"]):
        plt.scatter(psi_clean[labels == lab],
                    r2_clean[labels == lab],
                    s=12, c=col,
                    label=f"Cluster {lab}")
    plt.xlabel(r"$\psi$")
    plt.ylabel(r"$R^2$")
    plt.title("K‑Means‑Clustering (nach NaN‑Entfernung)")
    plt.legend()
    plt.grid(True, ls=":", alpha=0.5)
    plt.tight_layout()
    km_plot_path = PLOTS_DIR / "kmeans_cluster.png"
    plt.savefig(km_plot_path, dpi=300)
    plt.close()
    print(f"✅ Plot K‑Means‑Cluster gespeichert → {km_plot_path}")
else:
    print("⚠️ Zu wenige gültige Punkte für K‑Means – Clustering übersprungen.")
    labels = np.full_like(psi_clean, fill_value=-1)   # -1 = kein Cluster

# -------------------------------------------------
# 6️⃣  Ergebnis‑Export (CSV + JSON)
# -------------------------------------------------
# CSV – flache Tabelle, gut für Excel/Sheets
with CSV_OUT.open("w", newline="", encoding="utf-8") as csv_fp:
    writer = csv.writer(csv_fp)
    writer.writerow([
        "psi", "R2", "cluster_label", "fit_slope", "fit_intercept"
    ])
    for psi, r2, cl, sl, it in zip(psi_clean, r2_clean,
                                   labels, slope_clean, inter_clean):
        writer.writerow([psi, r2, cl, sl, it])

print(f"✅ CSV‑Export fertig → {CSV_OUT}")

# JSON – strukturierte Daten inkl. Metadaten
json_payload = {
    "metadata": agg.get("metadata", {}),
    "target_error": agg.get("target_error", "E2"),
    "theta_fixed": agg.get("theta_fixed", None),
    "poly_degree": agg.get("poly_degree", 1),
    "entries": []   # Liste von Dictionaries, ein Eintrag pro gültigem ψ
}

for psi, r2, cl, sl, it in zip(psi_clean, r2_clean,
                               labels, slope_clean, inter_clean):
    json_payload["entries"].append({
        "psi": float(psi),
        "R2": float(r2),
        "cluster": int(cl),
        "fit_slope": float(sl),
        "fit_intercept": float(it)
    })

with JSON_OUT.open("w", encoding="utf-8") as fp:
    json.dump(json_payload, fp, indent=2, ensure_ascii=False)

print(f"✅ JSON‑Export fertig → {JSON_OUT}")

# -------------------------------------------------
# 7️⃣  Abschlussmeldung
# -------------------------------------------------
print("\n🚀 Alle Analysen abgeschlossen!")
print(f"→ Plots befinden sich im Ordner: {PLOTS_DIR}")
print(f"→ CSV‑Ergebnisse: {CSV_OUT}")
print(f"→ JSON‑Ergebnisse: {JSON_OUT}")

Ursprünglich 40 Einträge → 37 gültige (ohne NaN).
✅ Plot R²‑Scatter gespeichert → analysis_output\plots\r2_scatter.png
✅ Plot K‑Means‑Cluster gespeichert → analysis_output\plots\kmeans_cluster.png
✅ CSV‑Export fertig → analysis_output\psi_results.csv
✅ JSON‑Export fertig → analysis_output\psi_results.json

🚀 Alle Analysen abgeschlossen!
→ Plots befinden sich im Ordner: analysis_output\plots
→ CSV‑Ergebnisse: analysis_output\psi_results.csv
→ JSON‑Ergebnisse: analysis_output\psi_results.json


In [4]:
"""
psi_analysis_2.py

- Lädt deine psi_results.json (Triple‑Scan‑Ergebnis).
- Berechnet Statistik pro Cluster.
- Findet den Switch‑ψ‑Wert (Übergang zwischen den Clustern).
- Erstellt zwei Plots (R²‑Scatter + Box‑Plot).
- Exportiert:
    * CSV  → analysis_output/psi_summary.csv
    * JSON → analysis_output/psi_summary.json   (alle Schlüssel sind native Python‑Typen)
"""

import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import csv

# -------------------------------------------------
# 1️⃣  Pfade & Vorbereitung
# -------------------------------------------------
INPUT_JSON = Path("analysis_output/psi_results.json")               # ← deine Datei
OUT_ROOT   = Path("analysis_output")                # Ausgabe‑Ordner
OUT_ROOT.mkdir(exist_ok=True)                       # Ordner anlegen, falls er fehlt

# -------------------------------------------------
# 2️⃣  Daten einlesen
# -------------------------------------------------
with INPUT_JSON.open("r", encoding="utf-8") as fp:
    raw = json.load(fp)

entries = raw["entries"]

psi   = np.array([e["psi"] for e in entries])
R2    = np.array([e["R2"] for e in entries])
cluster = np.array([e["cluster"] for e in entries])
slope = np.array([e["fit_slope"] for e in entries])
inter = np.array([e["fit_intercept"] for e in entries])

# -------------------------------------------------
# 3️⃣  Statistik pro Cluster (natürliche Python‑Typen)
# -------------------------------------------------
def stats_for_cluster(c):
    mask = cluster == c
    return {
        "count": int(mask.sum()),
        "R2_mean": float(R2[mask].mean()),
        "R2_median": float(np.median(R2[mask])),
        "slope_mean": float(slope[mask].mean()),
        "intercept_mean": float(inter[mask].mean())
    }

# Wir wandeln die Schlüssel (Cluster‑IDs) in Strings um,
# weil JSON‑Keys nur str/int/float/bool/None erlauben.
stats = {str(int(c)): stats_for_cluster(c) for c in np.unique(cluster)}

print("\n📊 Statistik pro Cluster")
for cid, s in stats.items():
    print(f"  Cluster {cid}: {s}")

# -------------------------------------------------
# 4️⃣  Switch‑ψ (erstes Auftreten eines Cluster‑Wechsels)
# -------------------------------------------------
# Der erste Index, an dem sich das Cluster ändert:
switch_idx = np.where(np.diff(cluster) != 0)[0][0] + 1   # +1 wegen diff‑Shift
psi_switch = psi[switch_idx]
print(f"\n🔎 Cluster‑Übergang bei ψ ≈ {psi_switch:.4f}")

# -------------------------------------------------
# 5️⃣  Plot 1: R² vs. ψ, farbig nach Cluster + Trend‑Linien
# -------------------------------------------------
plt.figure(figsize=(9, 5))
colors = {"0": "steelblue", "1": "orange"}

for cid in stats.keys():
    mask = (cluster == int(cid))
    plt.scatter(psi[mask], R2[mask], s=30,
                color=colors[cid], label=f"Cluster {cid} (n={mask.sum()})")

    # Linearer Trend pro Cluster (nur zur Visualisierung)
    coeff = np.polyfit(psi[mask], R2[mask], deg=1)
    psi_fit = np.linspace(psi.min(), psi.max(), 300)
    plt.plot(psi_fit, np.polyval(coeff, psi_fit),
             color=colors[cid], lw=2, ls="--",
             label=f"Fit Cluster {cid}")

plt.axvline(psi_switch, color="red", ls=":", lw=1.5,
            label=f"Switch ψ={psi_switch:.2f}")
plt.xlabel(r"$\psi$")
plt.ylabel(r"$R^2$")
plt.title("R²‑Qualität über das ψ‑Spektrum (farbig nach Cluster)")
plt.legend()
plt.grid(True, ls=":", alpha=0.6)
plt.tight_layout()
scatter_path = OUT_ROOT / "plots/r2_scatter2.png"
plt.savefig(scatter_path, dpi=300)
plt.close()
print(f"✅ Scatter‑Plot gespeichert → {scatter_path}")

# -------------------------------------------------
# 6️⃣  Plot 2: Box‑Plot der R²‑Verteilung pro Cluster
# -------------------------------------------------
plt.figure(figsize=(7, 4))
box_data = [R2[cluster == int(cid)] for cid in sorted(stats.keys())]
plt.boxplot(box_data,
            tick_labels=[f"Cluster {cid}" for cid in sorted(stats.keys())],
            patch_artist=True,
            boxprops=dict(facecolor="lightblue"))
plt.ylabel(r"$R^2$")
plt.title("Box‑Plot der R²‑Verteilung pro Cluster")
plt.grid(True, ls=":", alpha=0.5)
plt.tight_layout()
box_path = OUT_ROOT / "plots/r2_boxplot.png"
plt.savefig(box_path, dpi=300)
plt.close()
print(f"✅ Box‑Plot gespeichert → {box_path}")

# -------------------------------------------------
# 7️⃣  Export: CSV
# -------------------------------------------------
csv_path = OUT_ROOT / "psi_summary.csv"
with csv_path.open("w", newline="", encoding="utf-8") as fp:
    writer = csv.writer(fp)
    writer.writerow(["psi", "R2", "cluster", "fit_slope", "fit_intercept"])
    for row in zip(psi, R2, cluster, slope, inter):
        writer.writerow([float(row[0]), float(row[1]),
                        int(row[2]), float(row[3]), float(row[4])])
print(f"✅ CSV‑Datei geschrieben → {csv_path}")

# -------------------------------------------------
# 8️⃣  Export: JSON (alle Keys sind native Python‑Typen)
# -------------------------------------------------
json_out = OUT_ROOT / "psi_summary.json"
summary = {
    "cluster_stats": stats,               # schon konvertiert (String‑Keys)
    "psi_switch": float(psi_switch),
    "entries": [
        {
            "psi": float(p),
            "R2": float(r),
            "cluster": int(c),
            "fit_slope": float(s),
            "fit_intercept": float(i)
        }
        for p, r, c, s, i in zip(psi, R2, cluster, slope, inter)
    ]
}
with json_out.open("w", encoding="utf-8") as fp:
    json.dump(summary, fp, indent=2, ensure_ascii=False)
print(f"✅ JSON‑Datei geschrieben → {json_out}")

# -------------------------------------------------
# 9️⃣  Abschlussmeldung
# -------------------------------------------------
print("\n🚀 Analyse abgeschlossen! Alle Ergebnisse liegen im Ordner:")
print(f"   {OUT_ROOT.resolve()}")


📊 Statistik pro Cluster
  Cluster 0: {'count': 19, 'R2_mean': 0.0710609583945338, 'R2_median': 0.062304964692192155, 'slope_mean': 2.42684720610925e-17, 'intercept_mean': -3.579310256913258e-18}
  Cluster 1: {'count': 18, 'R2_mean': 0.08559560925263132, 'R2_median': 0.09273559354248279, 'slope_mean': 2.7774452083204964e-17, 'intercept_mean': -4.1486811945215746e-18}

🔎 Cluster‑Übergang bei ψ ≈ 5.6154
✅ Scatter‑Plot gespeichert → analysis_output\plots\r2_scatter2.png
✅ Box‑Plot gespeichert → analysis_output\plots\r2_boxplot.png
✅ CSV‑Datei geschrieben → analysis_output\psi_summary.csv
✅ JSON‑Datei geschrieben → analysis_output\psi_summary.json

🚀 Analyse abgeschlossen! Alle Ergebnisse liegen im Ordner:
   C:\Users\User\Python\V5\analysis_output
